In [33]:
# imports
import torch
from torch import nn
from torch.optim import SGD
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor, Normalize, Compose
from IPython.display import Image
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [2]:
# summary of neural networks
Image(url='https://i.imgur.com/ktUOnca.jpg')

# What is Deep Learning?

A machine learning strategy for learning representations (embeddings) of data.

* **Input:** An input vector for a piece of data you wish to represent.
* **Process:** *magic*
* **Output:** An embedding for your data.

# *magic* ?!?

Yes.

The process is a neural network. It applies a sequence of functions to the input:

* **Input:** $x$
* **Output:** $f_n(f_{n-1}(\dots f_2(f_1(x))\dots))$
* Traditionally, each $f_i$ is a matrix multiplication followed by a nonlinearity (called an *activation function*).
* Each fi can basically be anything as long as it is **differentiable**.
* We call these $f_i$’s *layers*.


## Example: Basic Linear Layer with ReLU activation function.

Let's let 
* $f_1(x) = \begin{bmatrix} 1 & 2 \\ 4 & 3 \end{bmatrix}x + \begin{bmatrix}0 \\ 1\end{bmatrix}$, 
* $f_2(x) = max(0, x)$,
* $x = \begin{bmatrix} 1 \\ -1 \end{bmatrix}$

So $f_2(f_1(x)) = f_2(\begin{bmatrix} -1 \\ 2\end{bmatrix})=\begin{bmatrix}0 \\ 2\end{bmatrix}$

* Here, $f_1$ is called a linear layer and $f_2$ is called the ReLU activation function.
* Linear layers needn't have the same number of inputs and outputs.
* Typically the parameters inside of the matrices of $f_1$ are *learned* in training over a dataset.
* This learning is done with backpropogation, and is the reason that we need our layers to be differentiable.

## idea behind backprop
* You have an input $x$ with a target output $t$.
* Pass the input $x$ through the series of layers to get an output $y=f_n(\dots f_1(x)\dots)$
* Calculate some error $e$ on $y$ with respect to the target output $t$ (e.g. from classification or regression).
* That error is called the *loss* and you want to learn parameters in your $f_i$'s that minimize it.
### **WARNING!!! MATH AHEAD**
* For each parameter $p$ in your $f_i$'s, you can calculate the derivative $\frac{de}{dp}$ via repeated use of the chain rule and use that derivative (called a gradient) to update the parameter $p$ to reduce your loss.
* this process is called *backpropogation*, since it moves backwards through the network.
* Since the derivative of a parameter is calculated with respect to the derivatives of all downstream parameters, any parameter that depends on a parameter which is learned needs a gradient.

### Practical notes:
* A loss is typically defined as the sum of the individual losses over the entire dataset.
* It is too expensive to compute the loss for the entire dataset before updating the parameters.
* Updating parameters for every single example tends to be too chaotic to get the model to converge.
* We instead process the data in random batches as a hybrid of the two approaches.

In [3]:
Image(url="http://www.frank-dieterle.de/phd/images/image016.gif")

Let's look at how this works in PyTorch

In [29]:
# how tensors work
x = torch.zeros(2,3)
x


 0  0  0
 0  0  0
[torch.FloatTensor of size 2x3]

In [8]:
# how variables work
x = Variable(x)
print "x:", x
print "requires grad:", x.requires_grad
print "data:", x.data

x: Variable containing:
 0  0  0
 0  0  0
[torch.FloatTensor of size 2x3]

requires grad: False
data: 
 0  0  0
 0  0  0
[torch.FloatTensor of size 2x3]



In [9]:
y = x + 1
z = Variable(torch.ones(2,3), requires_grad=True)
w = y+z

print y
print "does y require grad?", y.requires_grad
print "does w require grad?", w.requires_grad

Variable containing:
 1  1  1
 1  1  1
[torch.FloatTensor of size 2x3]

does y require grad? False
does w require grad? True


In [32]:
# what about neural network layers?
lin = nn.Linear(3, 4)  # 3 inputs and 4 outputs. Should contain a 4x3 matrix and a 4x1 matrix of parameters.
list(lin.parameters())

[Parameter containing:
 -0.5403  0.1461 -0.1120
  0.2643  0.2523 -0.5687
  0.3951 -0.4703  0.0070
  0.0127  0.4424  0.3770
 [torch.FloatTensor of size 4x3], Parameter containing:
  0.0275
 -0.2266
 -0.2756
  0.0264
 [torch.FloatTensor of size 4]]

In [11]:
print "lin output", lin(y)  # processing with a batch size of 2.
print lin(y).requires_grad
print lin(w).requires_grad

lin output Variable containing:
 0.3216  0.0921  0.3608 -1.2713
 0.3216  0.0921  0.3608 -1.2713
[torch.FloatTensor of size 2x4]

True
True


In [12]:
relu = nn.ReLU()
print relu(lin(y))
print relu(lin(y)).requires_grad
print relu(w).requires_grad
print relu(y).requires_grad

Variable containing:
 0.3216  0.0921  0.3608  0.0000
 0.3216  0.0921  0.3608  0.0000
[torch.FloatTensor of size 2x4]

True
True
False


# Kinds of data and their layers:
## Convolutional layers (good for image data).
* An $m\times n$ image with $3$ color channels is $m\times n \times 3$ dimensional. 
* For $m=n=200$, $m\times n \times 3 = 120000$, which is a lot of inputs for a linear layer to take in right away. Suppose you wanted to have an output dimension of 64. That would require $120000\times 64 + 64 = 7680064$ parameters.
* **Idea:** why not use the fact that patches of regions across the image have similar patterns? If we use this fact, then a lot of weights over the entire image can actually be the same.
* Let's focus on learning to get good with $3\times 3$ patches for now...
* A convolutional network over $3\times 3$ patches with the same number of inputs and outputs as above will have a set of $3 \times 3 \times 3 \times 64 + 64= 1792$ parameters.

In [13]:
Image(url="https://i.stack.imgur.com/GvsBA.jpg")

## Recurrent layers (good for sequential data).
* Suppose you have a training set of sentences for sentiment classification.
* Each sentence has either positive, negative, or neutral sentiment.
* How do you feed that through a model?
* Concatenate the word2vec word embeddings for the words together and use that as input?
* **Problem:** sentences are variable length. 
* You could just grab the first $N$ words of the sentence, but that probably won't do too well at detecting sentiment. Also, you'd have an input size of $N\times m$ where $m$ is the size of your word embeddings.
### Alternative approach: use a recurrent layer.
* Holds an internal *state* vector for the built up representation of the sequence up until that point.
* Could process word embeddings one at a time and learn to output the sentiment at the end.

In [14]:
Image(url="http://d3kbpzbmcynnmx.cloudfront.net/wp-content/uploads/2015/09/rnn.jpg")

In [15]:
# setting up datasets.
training = DataLoader(
    MNIST(
        "./train", 
        download=False, 
        transform=Compose([ToTensor(),Normalize((0.1307,), (0.3081,))])
    ), 
    batch_size=64, 
    shuffle=True, 
    num_workers=40
)
testing = DataLoader(
    MNIST(
        "./test", 
        train=False,
        download=False, 
        transform=Compose([ToTensor(),Normalize((0.1307,), (0.3081,))])
    ), 
    batch_size=64, 
    shuffle=True, 
    num_workers=40
)


In [16]:
len(training.dataset), len(testing.dataset)

(60000, 10000)

In [17]:
print MNIST('./train')[0]
MNIST('./train')[0][0]

(<PIL.Image.Image image mode=L size=28x28 at 0x7F511CBB4590>, 5)


In [18]:
# basic linear with relu's
class LinReLU(nn.Module):
    def __init__(self, indim, outdim):
        super(LinReLU, self).__init__()
        self.lin = nn.Linear(indim, outdim)
        self.relu = nn.ReLU(True)
    
    def forward(self, input):
        input = self.lin(input)
        return self.relu(input)


class MnistLinear(nn.Module):
    def __init__(self):
        super(MnistLinear, self).__init__()
        self.lin1 = LinReLU(28*28, 16)
        self.lin2 = nn.Linear(16, 10)
    
    def forward(self, input):
        input = self.lin1(input.view(-1, 28*28))
        input = self.lin2(input)
        return input

In [19]:
class ConvReLU(nn.Module):
    def __init__(self, in_channels, out_channels, kernel=3):
        super(ConvReLU, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel, padding=kernel/2)
        self.relu = nn.ReLU(True)
        self.bn = nn.BatchNorm2d(out_channels)
    
    def forward(self, input):
        input = self.conv(input)
        input = self.relu(input)
        return self.bn(input)


class MnistCNN(nn.Module):
    def __init__(self):
        super(MnistCNN, self).__init__()
        self.conv1 = ConvReLU(1, 16)
        self.mp = nn.MaxPool2d(2)
        self.conv2 = ConvReLU(16, 32)
        self.avgpool = nn.AvgPool2d(14)
        self.lin = nn.Linear(32, 10)
    
    def forward(self, input):
        input = self.conv1(input)
        input = self.mp(input)
        input = self.conv2(input)
        input = self.avgpool(input).squeeze()
        return self.lin(input)

In [20]:
class MnistRNN(nn.Module):
    def __init__(self):
        super(MnistRNN, self).__init__()
        self.lstm = nn.LSTM(
            input_size=28, 
            hidden_size=16, 
            num_layers=2, 
            batch_first=True, 
            dropout=0.5
        )
        self.linear = nn.Linear(16, 10)
    
    def forward(self, input):
        input, hidden = self.lstm(input.squeeze())
        return self.linear(input[:, -1])

In [21]:
class ComboModel(nn.Module):
    def __init__(self):
        super(ComboModel, self).__init__()
        self.lin = MnistLinear()
        self.cnn = MnistCNN()
        self.rnn = MnistRNN()
    
    def forward(self, input):
        l1 = self.lin(input)
        l2 = self.cnn(input)
        l3 = self.rnn(input)
        return l1, l2, l3 

In [22]:
model = ComboModel()

In [23]:
def num_parameters(m):
    return sum([y.nelement() for y in m.parameters()])

print "num params for linear:\t", num_parameters(model.lin)
print "num params for cnn:\t", num_parameters(model.cnn)
print "num params for rnn:\t", num_parameters(model.rnn)

num params for linear:	12730
num params for cnn:	5226
num params for rnn:	5290


In [24]:
optimizer = SGD(model.parameters(), lr=0.05, momentum=0.9)

In [25]:
model = model.cuda(1)

In [26]:
loss = nn.CrossEntropyLoss().cuda(1)

In [27]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(training):
        data, target = data.cuda(1), target.cuda(1)
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        lin_out, cnn_out, rnn_out = model(data)
        lin_loss = loss(lin_out, target)
        cnn_loss = loss(cnn_out, target)
        rnn_loss = loss(rnn_out, target)
        total_loss = lin_loss + cnn_loss + rnn_loss
        total_loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLin Loss: {:.6f}\tCNN Loss: {:.6f}\tRNN Loss: {:.6f}'.format(
                epoch+1, batch_idx, len(training),
                100. * batch_idx / len(training), lin_loss.data[0], cnn_loss.data[0], rnn_loss.data[0]))

num_epochs = 10
losses = [{}] * (num_epochs+1)
correct = [{}] * (num_epochs+1)

def test(epoch):
    model.eval()
    losses[epoch] = dict(
        Linear=0,
        CNN=0,
        RNN=0
    )
    correct[epoch] = dict(
        Linear=0,
        CNN=0,
        RNN=0
    )
    for data, target in testing:
        data, target = data.cuda(1), target.cuda(1)
        data, target = Variable(data, volatile=True), Variable(target)
        lin_out, cnn_out, rnn_out = model(data)
        
        for name, output in [("Linear", lin_out), ("CNN", cnn_out), ("RNN", rnn_out)]:
            losses[epoch][name] += loss(output, target).data[0]
            pred = output.data.max(1)[1] 
            correct[epoch][name] += pred.eq(target.data).cpu().sum()

    for name in ['Linear', 'CNN', 'RNN']:
        losses[epoch][name] /= len(testing) # loss function already averages over batch size
        print('{} Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            name, losses[epoch][name], correct[epoch][name], len(testing.dataset),
            100. * correct[epoch][name] / len(testing.dataset)))
        
for epoch in range(num_epochs+1):
    test(epoch)
    train(epoch)
test(num_epochs)

Linear Test set: Average loss: 2.3522, Accuracy: 1105/10000 (11%)

CNN Test set: Average loss: 2.3099, Accuracy: 974/10000 (10%)

RNN Test set: Average loss: 2.3116, Accuracy: 974/10000 (10%)

Train Epoch: 1 [0/938 (0%)]	Lin Loss: 2.308277	CNN Loss: 2.316426	RNN Loss: 2.308771
Train Epoch: 1 [100/938 (11%)]	Lin Loss: 1.472899	CNN Loss: 1.754471	RNN Loss: 2.240767
Train Epoch: 1 [200/938 (21%)]	Lin Loss: 0.468504	CNN Loss: 0.889403	RNN Loss: 1.705237
Train Epoch: 1 [300/938 (32%)]	Lin Loss: 0.266181	CNN Loss: 0.397529	RNN Loss: 1.179084
Train Epoch: 1 [400/938 (43%)]	Lin Loss: 0.140255	CNN Loss: 0.269138	RNN Loss: 0.977627
Train Epoch: 1 [500/938 (53%)]	Lin Loss: 0.233862	CNN Loss: 0.236546	RNN Loss: 0.647767
Train Epoch: 1 [600/938 (64%)]	Lin Loss: 0.182968	CNN Loss: 0.215670	RNN Loss: 0.458684
Train Epoch: 1 [700/938 (75%)]	Lin Loss: 0.280577	CNN Loss: 0.170136	RNN Loss: 0.435829
Train Epoch: 1 [800/938 (85%)]	Lin Loss: 0.171542	CNN Loss: 0.073429	RNN Loss: 0.326413
Train Epoch: 1 [90

In [28]:
scatters = []
for name in ['Linear', 'CNN', 'RNN']:
    xs = [i for i in range(1, num_epochs+1)]
    ys = [float(i[name])/len(testing.dataset) for i in correct]
    scatters.append(go.Scatter(
        x = xs,
        y = ys,
        name=name
    ))
    print ys
iplot(scatters)

[0.1105, 0.92, 0.9271, 0.9267, 0.9288, 0.9323, 0.9275, 0.9292, 0.9378, 0.9323, 0.9357]
[0.0974, 0.9371, 0.9387, 0.9711, 0.9645, 0.9721, 0.9783, 0.978, 0.9689, 0.9771, 0.9809]
[0.0974, 0.8913, 0.9445, 0.9477, 0.9479, 0.9623, 0.9579, 0.9576, 0.9653, 0.9668, 0.9621]


http://pytorch.org/docs/